In [ ]:
library(dplyr)
library(repr)
library(tidyverse)
library(tidymodels)
library(GGally)

In [ ]:
algerian_fire_0 <- read.csv(url("https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv"), skip = 1)
algerian_fire <- algerian_fire_0 %>% #set up right labels 
     select(day, month, year, Classes,Temperature, RH, Ws, Rain) %>%
    mutate(Temperature = as.numeric(Temperature)) %>%
    mutate(RH = as.numeric(RH)) %>%
    mutate(Ws = as.numeric(Ws)) %>%
    mutate(Rain = as.numeric(Rain)) 
algerian_fire$date <- as.Date(with(algerian_fire, paste(year, month, day, sep="-")), "%Y-%m-%d") #combine three columns and set it as date
tidy_al <- algerian_fire %>% select(date, Classes,Temperature, RH, Ws, Rain) #tidy data for the whole dataset

In [ ]:
bejaia_fire <- tidy_al[c(1:122),] #select data for bejaia region
bejaia_fire$Classes <- trimws(bejaia_fire$Classes) #remove space before and after the string
bejaia_fire <- bejaia_fire %>% mutate(Classes = as.factor(Classes)) #set the label back to factor
bejaia_split <- initial_split(bejaia_fire, prop = 0.8, strata = Classes)
bejaia_train <- training(bejaia_split) #split and get training dataset
mean_of_variable <- map_df(bejaia_train, mean, na.rm=TRUE)
mean_of_variable

In [ ]:
summary(with(bejaia_train, Classes=="fire"))
nrow(bejaia_train)

In [ ]:
n_of_observations <- bejaia_train$Classes %>% map_lgl("fire", .default=)
n_of_observations

In [ ]:
?as_mapper()

In [ ]:
tibble("Number of obervations"=99, "Fire"=51, "Not Fire"=48 ) #create table for number of observations for each classes

In [ ]:
#not sure if this is a good visulization
#options(repr.plot.width = 10, repr.plot.height = 10)
#pairs_bejaia <- bejaia_train %>%
#  select(Temperature, RH, Ws, Rain) %>%
#  ggpairs(ggplot2::aes(colour=bejaia_train$Classes)) 
#pairs_bejaia

In [ ]:
bejaia_recipe <- recipe(Classes~Temperature+RH+Ws+Rain, data=bejaia_train) %>% #scale the data
    step_center(all_predictors()) %>%
    step_scale(all_predictors())
bejaia_scaled <- bejaia_recipe %>%  
    prep() %>%
    bake(bejaia_train)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)
b_RH_plot <- bejaia_scaled %>% 
    ggplot(aes(y=Temperature,x=RH, color=Classes)) + 
    geom_point() + 
    labs(y="Temperature", x="Standardized Relative Humidity", colour = "Classes")+
    theme(text = element_text(size = 20))
b_Ws_plot <- bejaia_scaled %>% 
    ggplot(aes(y=Temperature, x=Ws, color=Classes)) + 
    geom_point() + 
    labs(y="Temperature",x="Standardized Wind Spead", colour = "Classes") +
    theme(text = element_text(size = 20))
b_RH_plot 
b_Ws_plot